In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
from matplotlib.colors import Normalize
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
from matplotlib.ticker import MultipleLocator

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### Reading hdf5 data

In [ ]:
with h5py.File(f'/home/erick/software/devscrpts/gw170817_paper_plots/1506_ppEb_0.32ms_sigmaFFI_slices.h5', 'r') as f:

    xz_x_km_ffi = f['xz_x_km'][:]
    xz_y_km_ffi = f['xz_y_km'][:]
    xz_z_invsec_ffi = f['xz_z_invsec'][:]

    xy_x_km_ffi = f['xy_x_km'][:]
    xy_y_km_ffi = f['xy_y_km'][:]
    xy_z_invsec_ffi = f['xy_z_invsec'][:]


with h5py.File(f'/home/erick/software/devscrpts/gw170817_paper_plots/1506_ppEb_0.32ms_sigmaCFI_slices.h5', 'r') as f:

    xz_x_km_cfi = f['xz_x_km'][:]
    xz_y_km_cfi = f['xz_y_km'][:]
    xz_z_invsec_cfi = f['xz_z_invsec'][:]

    xy_x_km_cfi = f['xy_x_km'][:]
    xy_y_km_cfi = f['xy_y_km'][:]
    xy_z_invsec_cfi = f['xy_z_invsec'][:]

In [ ]:
bh_r=5.43, # km
bh_x=48.0, # km
bh_y=48.0, # km
bh_z=16.0, # km

In [ ]:
cmap = 'plasma'

# profile slices
temp_slice_up  = np.array(xy_z_invsec_ffi)
ye_slice_up    = np.array(xy_z_invsec_cfi)
temp_slice_low = np.array(xz_z_invsec_ffi)
ye_slice_low   = np.array(xz_z_invsec_cfi)

xup = np.array(xy_x_km_ffi)
yup = np.array(xy_y_km_ffi)
xlow = np.array(xz_x_km_ffi)
ylow = np.array(xz_y_km_ffi)

# color bar limits
temp_max = np.max([np.nanmax(temp_slice_low), np.nanmax(temp_slice_up)])
temp_min = np.min([np.nanmin(temp_slice_low), np.nanmin(temp_slice_up)])
ye_max = np.max([np.nanmax(ye_slice_low), np.nanmax(ye_slice_up)])
ye_min = np.min([np.nanmin(ye_slice_low), np.nanmin(ye_slice_up)])
xmid = 96 / 2 # km
ymid = 96 / 2
zmid = 32 / 2 # km

# create 2x2 figure
fig, axes = plt.subplots(2, 2, figsize=(12, 12), sharex='col', sharey='row')
plt.subplots_adjust(wspace=0, hspace=-0.19)
fig.align_labels()

import mpl_toolkits

# colorbars
norm = mpl.colors.Normalize(vmin=temp_min, vmax=temp_max)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Required for ScalarMappable even if unused
divider = mpl_toolkits.axes_grid1.make_axes_locatable(axes[0,0])
cax = divider.append_axes("top", size="8%", pad=0.15)  # <-- Increase pad value to move colorbar up
cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
cbar.ax.tick_params(which="both",direction="in")
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())
cbar.set_label(r"$\log \sigma_{\mathrm{FFI}} \,[1/\mathrm{s}]$", labelpad=10)
cax.xaxis.set_major_locator(MultipleLocator(1))

norm = mpl.colors.Normalize(vmin=ye_min, vmax=ye_max)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Required for ScalarMappable even if unused
divider = mpl_toolkits.axes_grid1.make_axes_locatable(axes[0,1])
cax = divider.append_axes("top", size="8%", pad=0.15)  # <-- Increase pad value here too
cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
cbar.ax.tick_params(which="both",direction="in")
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())
cbar.set_label(r"$\log \sigma_{\mathrm{CFI}} \,[1/\mathrm{s}]$", labelpad=10)

# set axis labels
for ax in axes[-1,:]:
    ax.set_xlabel(r'$x\,[\mathrm{km}]$')
axes[0,0].set_ylabel(r'$z\,[\mathrm{km}]$')
axes[1,0].set_ylabel(r'$y\,[\mathrm{km}]$')
# remove xtick labels on all but the bottom row
for ax in axes[:-1,:].flatten():
    for label in ax.get_xticklabels():
        label.set_visible(False) 
for ax in axes[:,1:].flatten():
    for label in ax.get_yticklabels():
        label.set_visible(False)

for ax in axes.flat:
    ax.set_aspect('equal')
    ax.xaxis.set_major_locator(MultipleLocator(20))
    ax.yaxis.set_major_locator(MultipleLocator(20))
    # ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
    ax.minorticks_on()
    # pf.apply_custom_settings(ax, False)

# mask out the black hole region
axes[0,0].pcolormesh(xlow-xmid, ylow-zmid, temp_slice_low, shading='auto', cmap=cmap, vmin=temp_min, vmax=temp_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,0].add_patch(circle_bh)

axes[1,0].pcolormesh(xup-xmid , yup-ymid , temp_slice_up , shading='auto', cmap=cmap, vmin=temp_min, vmax=temp_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,0].add_patch(circle_bh)
# Add legend for black hole horizon
legend_circle = Line2D([0], [0], color='k', linestyle='dashed', linewidth=1, label='Black hole')
axes[1,0].legend(handles=[legend_circle], loc='upper right', frameon=False, fontsize=14)

axes[0,1].pcolormesh(xlow-xmid, ylow-zmid, ye_slice_low  , shading='auto', cmap=cmap, vmin=ye_min  , vmax=ye_max  )
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,1].add_patch(circle_bh)

axes[1,1].pcolormesh(xup-xmid , yup-ymid , ye_slice_up   , shading='auto', cmap=cmap, vmin=ye_min  , vmax=ye_max  )
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,1].add_patch(circle_bh)

plt.savefig('plots/global_instabilities.pdf', bbox_inches='tight')



In [ ]:
energybinsMeV = [1.0, 3.0, 5.2382, 8.0097, 11.442, 15.691, 20.953, 27.468, 35.536, 45.525, 57.895, 73.212, 92.178]
f_nu_e = [0.006521702881797871, 0.020073470976539026, 0.03484658072278011, 0.03965698261033323, 0.02751185667432394, 0.011607702312938681, 0.003158601098696683, 0.0005733842698907312, 6.747931038641904e-05, 4.759529082699893e-06, 1.7909033181414476e-07, 3.0889951802233437e-09, 2.0201687097226764e-11]
f_nubar_e = [0.0008266765812236383, 0.0020606930415842904, 0.006136076070410455, 0.008123901263370346, 0.006917333469660352, 0.004069698424592733, 0.0016455337701198184, 0.00043876723841825493, 7.217538618235897e-05, 6.674620204193405e-06, 3.0766583031388274e-07, 6.093104719325275e-09, 4.3427656496137066e-11]
f_nu_x = [7.56497963607975e-05, 7.474592240119204e-05, 7.108862210033428e-05, 5.6367780201784625e-05, 3.639004905522398e-05, 1.8462764440419687e-05, 7.170408801806545e-06, 2.0754930776738156e-06, 4.4008978218765684e-07, 6.669263165682613e-08, 7.0687045361239646e-09, 5.08765057270609e-10, 2.353213998752438e-11]
feq_nu_e = [0.38779041356182936, 0.27250827875021494, 0.17224153032476175, 0.09130604563658595, 0.03919109191965831, 0.013185337513924523, 0.0033431129574536117, 0.0006055625354973532, 7.278674983430775e-05, 5.27970863082672e-06, 2.048960058438477e-07, 3.6667627949045053e-09, 2.516407424471708e-11]
feq_nubar_e = [0.48282818133551053, 0.35570882710799123, 0.23470565870571045, 0.1289925379160776, 0.05670959870084598, 0.019312875768383034, 0.0049195388384934745, 0.0008922677722069464, 0.0001072749112475792, 7.781614023131328e-06, 3.0199118515001655e-07, 5.4043520425354466e-09, 3.7088713910433024e-11]
feq_nu_x = [0.43470944727126204, 0.3126012900453216, 0.2016721507869382, 0.10872379785240406, 0.047183486234802674, 0.01596238441268892, 0.004055746637536582, 0.0007350774860911019, 8.836412259007845e-05, 6.409732026886588e-06, 2.4875045370427275e-07, 4.451570172811366e-09, 3.05500106462221e-11]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(energybinsMeV, np.log10(f_nu_e),    marker='o', label=r'$f_{     \nu_e}$')
ax.plot(energybinsMeV, np.log10(f_nubar_e), marker='o', label=r'$f_{\bar{\nu}_e}$')
ax.plot(energybinsMeV, np.log10(f_nu_x),    marker='o', label=r'$f_{     \nu_\mu}$')

ax.plot(energybinsMeV, np.log10(feq_nu_e),    marker='', label=r'$f_{     \nu_e}^{\mathrm{eq}}$',  color='C0', linestyle='dotted')
ax.plot(energybinsMeV, np.log10(feq_nubar_e), marker='', label=r'$f_{\bar{\nu}_e}^{\mathrm{eq}}$', color='C1', linestyle='dotted')
ax.plot(energybinsMeV, np.log10(feq_nu_x),    marker='', label=r'$f_{     \nu_\mu}^{\mathrm{eq}}$',  color='C2', linestyle='dotted')

print('energybinsMeV =', list(energybinsMeV))
print('f_nu_e =', list(f_nu_e))
print('f_nubar_e =', list(f_nubar_e))
print('f_nu_x =', list(f_nu_x))
print('feq_nu_e =', list(feq_nu_e))
print('feq_nubar_e =', list(feq_nubar_e))
print('feq_nu_x =', list(feq_nu_x))

ax.set_xlabel(r'$E\,[\mathrm{MeV}]$')
ax.set_ylabel(r'$\log f$')
leg = ax.legend(framealpha=0.0, ncol=2, fontsize=22)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
ax.set_xscale('log')
plt.savefig(f"plots/distribution_function.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)